In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import EXPERIMENT_NAME

RSEED = 42

# Modeling Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.dummy import DummyClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm


warnings.filterwarnings('ignore')


In [2]:
TRACKING_URI = open("../.mlflow_uri").read().strip()

In [3]:
df_features = pd.read_csv('../data/Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Training_Features.csv')

In [4]:
df_features.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [5]:
df_target = pd.read_csv('../data/Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Training_Labels.csv')

In [6]:
# Get info for the target
df_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   respondent_id     26707 non-null  int64
 1   h1n1_vaccine      26707 non-null  int64
 2   seasonal_vaccine  26707 non-null  int64
dtypes: int64(3)
memory usage: 626.1 KB


In [7]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                26707 non-null  int64  
 1   h1n1_concern                 26615 non-null  float64
 2   h1n1_knowledge               26591 non-null  float64
 3   behavioral_antiviral_meds    26636 non-null  float64
 4   behavioral_avoidance         26499 non-null  float64
 5   behavioral_face_mask         26688 non-null  float64
 6   behavioral_wash_hands        26665 non-null  float64
 7   behavioral_large_gatherings  26620 non-null  float64
 8   behavioral_outside_home      26625 non-null  float64
 9   behavioral_touch_face        26579 non-null  float64
 10  doctor_recc_h1n1             24547 non-null  float64
 11  doctor_recc_seasonal         24547 non-null  float64
 12  chronic_med_condition        25736 non-null  float64
 13  child_under_6_mo

In [8]:
df_features.isnull().sum(axis = 0)

respondent_id                      0
h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

- express missing values as %

Options:
- modelling for imputation (withput using our target variable!)
- or use prediction models that don't care about missing values
- or impute so that the overall distribution stays the same (based on statistics of this data set)
- or impute using a hypothesis (e.g. people who have missing values don't have health insurance?)
- for imputation we can try several approaches and see what gives best results :)

- remember--when we impute, we want to base assumptions based on the test set of a train-test split:
    - if we do multiple models, test-train split for each
    - create functions for imputation

- remember modelling for understanding (EDA style) =/= modelling for prediction

library for visualising missing values:
https://github.com/ResidentMario/missingno

In [9]:
# We are concatenating both dataframes into one 

df = pd.merge(df_target, df_features, on=['respondent_id'])

We will drop the following columns for our first iteration:    
- health_insurance, employment_industry, employment_occupation, income_poverty, marital_status, employment_status

What are the values in the features that have a lot of missing data?

In [10]:
df_features.health_insurance.value_counts()

1.0    12697
0.0     1736
Name: health_insurance, dtype: int64

Binary variable; 12% do not have health insurance, the rest do

In [11]:
df_features.employment_industry.value_counts()

fcxhlnwr    2468
wxleyezf    1804
ldnlellj    1231
pxcmvdjn    1037
atmlpfrs     926
arjwrbjb     871
xicduogh     851
mfikgejo     614
vjjrobsf     527
rucpziij     523
xqicxuve     511
saaquncn     338
cfqqtusy     325
nduyfdeo     286
mcubkhph     275
wlfvacwt     215
dotnnunm     201
haxffmxo     148
msuufmds     124
phxvnwax      89
qnlwzans      13
Name: employment_industry, dtype: int64

Anonymised variable with 21 values

In [12]:
df_features.employment_occupation.value_counts()

xtkaffoo    1778
mxkfnird    1509
emcorrxb    1270
cmhcxjea    1247
xgwztkwe    1082
hfxkjkmi     766
qxajmpny     548
xqwwgdyp     485
kldqjyjy     469
uqqtjvyb     452
tfqavkke     388
ukymxvdu     372
vlluhbov     354
oijqvulv     344
ccgxvspp     341
bxpfxfdn     331
haliazsg     296
rcertsgn     276
xzmlyyjv     248
dlvbwzss     227
hodpvpew     208
dcjcmpih     148
pvmttkik      98
Name: employment_occupation, dtype: int64

Anonymised variable with 23 values

# Data cleaning

Dropping of features with too many missing values:

In [13]:
col_drop = ['health_insurance', 'employment_industry', 'employment_occupation', 'income_poverty', 'marital_status', 'employment_status']

df.drop(col_drop, axis=1, inplace=True)

Dropping of all rows with null values:

In [14]:
df.dropna(inplace=True)

Check that all null values have been dropped:

In [15]:
df.isnull().sum(axis = 0)

respondent_id                  0
h1n1_vaccine                   0
seasonal_vaccine               0
h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
rent_or_own                    0
hhs_geo_region                 0
census_msa                     0
household_

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21853 entries, 0 to 26706
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                21853 non-null  int64  
 1   h1n1_vaccine                 21853 non-null  int64  
 2   seasonal_vaccine             21853 non-null  int64  
 3   h1n1_concern                 21853 non-null  float64
 4   h1n1_knowledge               21853 non-null  float64
 5   behavioral_antiviral_meds    21853 non-null  float64
 6   behavioral_avoidance         21853 non-null  float64
 7   behavioral_face_mask         21853 non-null  float64
 8   behavioral_wash_hands        21853 non-null  float64
 9   behavioral_large_gatherings  21853 non-null  float64
 10  behavioral_outside_home      21853 non-null  float64
 11  behavioral_touch_face        21853 non-null  float64
 12  doctor_recc_h1n1             21853 non-null  float64
 13  doctor_recc_seas

In [17]:
df.reset_index(inplace=True)

We dropped all rows with missing values
- Maybe later on, we will want to refine this approach.

In [18]:
 # We are looking for unique values in order to identify whether we have duplicates

df['respondent_id'].nunique()

21853

All values are unique, no duplicates. 

## EDA

In [19]:
# checking for balance in data

print(df.h1n1_vaccine.value_counts())
print(df.seasonal_vaccine.value_counts())

0    16906
1     4947
Name: h1n1_vaccine, dtype: int64
0    11371
1    10482
Name: seasonal_vaccine, dtype: int64


23% vaccination rate for H1N1 and 52% vaccination rate for seasonal flu
The H1N1 vaccine outcome appears to be unbalanced (almost 17k vs 5k) (read lit--what is considered unbalanced?)
The seasonal vaccine outcome appears to be fairly balanced

We may want to deal with the lack of balance later

In [20]:
# Understanding observations and rows

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21853 entries, 0 to 21852
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   index                        21853 non-null  int64  
 1   respondent_id                21853 non-null  int64  
 2   h1n1_vaccine                 21853 non-null  int64  
 3   seasonal_vaccine             21853 non-null  int64  
 4   h1n1_concern                 21853 non-null  float64
 5   h1n1_knowledge               21853 non-null  float64
 6   behavioral_antiviral_meds    21853 non-null  float64
 7   behavioral_avoidance         21853 non-null  float64
 8   behavioral_face_mask         21853 non-null  float64
 9   behavioral_wash_hands        21853 non-null  float64
 10  behavioral_large_gatherings  21853 non-null  float64
 11  behavioral_outside_home      21853 non-null  float64
 12  behavioral_touch_face        21853 non-null  float64
 13  doctor_recc_h1n1

In [21]:
#columns to drop because they're little use
col_drop = ['index', 'respondent_id']

df.drop(col_drop, axis=1, inplace=True)

- our target variables are h1n1_vaccine and seasonal_vaccine
- at the moment we are working with 28 feature variables--all categorical (refer to challenge documentation for description; we will need to transfer this info to the README)
- seven of the variables are strings--we will convert these to numeric encoding so we can look at correlations in Profiler
household_adults and household_children are 'top-coded' up to 3--that means that household with 3+ adults (or children) will fall into the '3' group
- 'hhs_geo_region' is an anonymised string
- we should remember that the current column names (which would be used as labels in the graphs) are not really human-readable--we need to keep this in mind when we're making plots (either rename the columns beforehand, or include a plotting command to change the labels)

24  age_group                    21853 non-null  object 
 25  education                    21853 non-null  object 
 26  race                         21853 non-null  object 
 27  sex                          21853 non-null  object 
 28  rent_or_own                  21853 non-null  object 
 29  hhs_geo_region               21853 non-null  object 
 30  census_msa                   21853 non-null  object 

24  age_group                    21853 non-null  object 
 25  education                    21853 non-null  object 
 26  race                         21853 non-null  object 
 27  sex                          21853 non-null  object 
 28  rent_or_own                  21853 non-null  object 
 29  hhs_geo_region               21853 non-null  object 
 30  census_msa                   21853 non-null  object 

Conversion of string variables to numeric (so these variables het displayed in Profiler properly) vie manual numeric encoding:

In [22]:
#a separate dataframe is made for the Profiler; the original dataframe will be retained for one-hot encoding (so the column headings we get during one-hot encoding remain meaningful)
df["age_group"].value_counts()

65+ Years        5393
55 - 64 Years    4655
45 - 54 Years    4390
18 - 34 Years    4277
35 - 44 Years    3138
Name: age_group, dtype: int64

In [23]:
df["education"].value_counts()

College Graduate    8839
Some College        6123
12 Years            4963
< 12 Years          1928
Name: education, dtype: int64

In [24]:
df["race"].value_counts()

White                17485
Black                 1670
Hispanic              1428
Other or Multiple     1270
Name: race, dtype: int64

In [25]:
df["sex"].value_counts()

Female    13105
Male       8748
Name: sex, dtype: int64

In [26]:
df["rent_or_own"].value_counts()

Own     16647
Rent     5206
Name: rent_or_own, dtype: int64

In [27]:
df["hhs_geo_region"].value_counts()

lzgpxyit    3481
fpwskwrf    2626
qufhixun    2588
bhuqouqj    2373
oxchjgsf    2356
kbazzjca    2299
mlyzmhmf    1832
atmpeygn    1694
lrircsnp    1686
dqpwygqj     918
Name: hhs_geo_region, dtype: int64

In [28]:
df["census_msa"].value_counts()

MSA, Not Principle  City    9558
MSA, Principle City         6362
Non-MSA                     5933
Name: census_msa, dtype: int64

In [29]:
cleanup = {"age_group": {"18 - 34 Years": 1, "35 - 44 Years": 2, "45 - 54 Years": 3, "55 - 64 Years": 4,
                                  "65+ Years": 5},
            "education": {"< 12 Years": 1, "12 Years": 2, "Some College": 3, "College Graduate": 4},
            "race": {"White": 1, "Black": 2, "Hispanic": 3, "Other or Multiple": 4},
            "sex" : {"Female": 1, "Male": 2},
            "rent_or_own" : {"Own": 1, "Rent": 2},
            "hhs_geo_region" : {"lzgpxyit": 1, "fpwskwrf": 2, "qufhixun": 3, "bhuqouqj": 4, "oxchjgsf": 5, "kbazzjca": 6, "mlyzmhmf": 7, "atmpeygn": 8, "lrircsnp": 9, "dqpwygqj": 10},
            "census_msa" : {"MSA, Not Principle  City": 1, "MSA, Principle City": 2, "Non-MSA": 3}
                                  }

In [30]:
df_for_profiler = df.replace(cleanup)
df_for_profiler.head()

,h1n1_vaccine,seasonal_vaccine,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,...,opinion_seas_sick_from_vacc,age_group,education,race,sex,rent_or_own,hhs_geo_region,census_msa,household_adults,household_children
0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2.0,4,1,1,1,1,5,3,0.0,0.0
1,0,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,...,4.0,2,2,1,2,2,4,1,0.0,0.0
2,0,1,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,1.0,5,2,1,1,2,9,2,0.0,0.0
3,0,0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,4.0,3,3,1,1,1,3,1,1.0,0.0
4,0,0,3.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,4.0,5,2,1,2,1,8,2,2.0,3.0


Run Profiler to explore the data:

In [31]:
#import Profiler
#before opening VS Code, run this command in the terminal: pip install pandas-profiling==2.11.0
from pandas_profiling import ProfileReport

In [32]:
profile = ProfileReport(df_for_profiler, title="Pandas Profiling Report", explorative=True)

In [33]:
#profile

**Possible multicollinearity (between features) based on heatmap:**
- behavioral_large_gatherings vs behavioral_outside_home
- doctor_recc_h1n1 vs doctor_recc_seasonal
- opinion_h1n1_risk vs opinion_seas_risk
- household_children vs age_group

**Possible outliers and features to be aware of:**
- behavioral_antiviral_meds : very unbalanced between categories; the people taking anviral meds could have something else going on (e.g. already sick, or worried about getting flu and taking meds profilactically)--be careful about this variable
- behavioral_face_mask
- behavioral_wash_hands
-child_under_6_months
- health_worker
- opinion_h1n1_vacc_effective and opinion_seasonal_vacc_effective (1.0 group)
- race (not many non-white respondents)
- household_adults and household_children (3.0 groups pretty small)

## Creating Pipelines

In [34]:
# Pipeline for categorical features
#CHECK HERE--ARE WE DROPPING THE FIRST COLUMN TO PREVENT MULTICOLINEARITY?
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown='error', drop='first'))
])

In [35]:
cat_features = list(df.columns)


Removal of target variables from cat_features list:

In [36]:
cat_features.remove('h1n1_vaccine')

In [37]:
cat_features.remove('seasonal_vaccine')

Rename the features and target to 'X' and 'y', to make the test-train split easier:

In [38]:
y = df[['h1n1_vaccine', 'seasonal_vaccine']].copy()

In [39]:
y = y.to_numpy()
y

array([[0, 0],
       [0, 1],
       [0, 1],
       ...,
       [0, 0],
       [0, 1],
       [0, 0]])

In [40]:

#NB: the H1N1 vaccine and seasonal vaccine are left in, otherwise the pipeline doesn't run properly
#X = df

#NB: dropping the 'h1n1_vaccine' and 'seasonal_vaccine' columns
X = df.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])

In [41]:
preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_features)
])

In [42]:
estimators = MultiOutputClassifier(
    estimator=LogisticRegression()#(penalty="l2", C=1)
)


In [43]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

In [44]:

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.2, random_state=RSEED)

In [45]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (17482, 29)
X_test shape: (4371, 29)
y_train shape: (17482, 2)
y_test shape: (4371, 2)


In [46]:
full_pipeline.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('1hot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['h1n1_concern',
                                                   'h1n1_knowledge',
                                                   'behavioral_antiviral_meds',
                                                   'behavioral_avoidance',
                                                   'behavioral_face_mask',
                                                   'behavioral_wash_hands',
                                                   'behavioral_large_gatherings',
                                                   'behavioral_outside_home',
                                                   'behavioral_touch_face',
                                                   'doctor

In [47]:
# Figure out later what this does and if we want to use it


#y_train_predicted = cross_val_predict(full_pipeline, X_train, y_train, cv=5)

In [48]:
preds = full_pipeline.predict(X_test)


Model evaluation

In [49]:
# Evaluation Metrices for H1N1 Vaccines
print("Accuracy: {:.2f}".format(accuracy_score(y_test[:, 0], preds[:, 0])))
print("Recall: {:.2f}".format(recall_score(y_test[:, 0], preds[:, 0])))
print("Precision: {:.2f}".format(precision_score(y_test[:, 0], preds[:, 0])))
print("F1: {:.2f}".format(f1_score(y_test[:, 0], preds[:, 0])))
print("ROC: {:.2f}".format(roc_auc_score(y_test[:, 0], preds[:, 0])))

Accuracy: 0.83
Recall: 0.44
Precision: 0.67
F1: 0.54
ROC: 0.69


In [50]:
# Evaluation Metrices for Seasonal Flu Vaccines
print("train data: {:.2f}".format(accuracy_score(y_test[:, 1], preds[:, 1])))
print("Recall: {:.2f}".format(recall_score(y_test[:, 1], preds[:, 1])))
print("Precision: {:.2f}".format(precision_score(y_test[:, 1], preds[:, 1])))
print("F1: {:.2f}".format(f1_score(y_test[:, 1], preds[:, 1])))
print("ROC: {:.2f}".format(roc_auc_score(y_test[:, 1], preds[:, 1])))

train data: 0.78
Recall: 0.74
Precision: 0.79
F1: 0.76
ROC: 0.78


In [51]:
y_test[:, 0]

array([1, 0, 0, ..., 0, 0, 0])

In [52]:
y_test

array([[1, 1],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 1],
       [0, 0]])

ROC is chosen for the following reasons:
1. curve consideres both -ves and +ves 
2. AUC_score tells how well model distinquishes between -ves and +ves 
3. Both outcomes are valuable because there is nor preference for either
4. For further reading https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc


The goal for the AUC_score is pegged at 0.8 based of the challenge data and the 
benchmarks reached i the competition https://www.researchgate.net/post/What-is-the-value-of-the-area-under-the-roc-curve-AUC-to-conclude-that-a-classifier-is-excellent

## BASELINE MODEL RESULTS

In [53]:
dummy_classifier = DummyClassifier()
dummy_classifier.fit(X_train, y_train)


DummyClassifier()

In [54]:
dummy_train_pred = dummy_classifier.predict(X_train)
dummy_test_pred = dummy_classifier.predict(X_test)

In [55]:
print("ROC: {:.2f}".format(roc_auc_score(y_test[:, 0], dummy_test_pred[:, 0])))
print("ROC: {:.2f}".format(roc_auc_score(y_test[:, 1], dummy_test_pred[:, 1])))

ROC: 0.50
ROC: 0.50


## Trainining the model and tracking with MLFlow

In [56]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run()
run = mlflow.active_run()

In [57]:
print("Active run_id: {}".format(run.info.run_id))

Active run_id: 9aeea82b093c45479c462b814aa00633


In [58]:
#training the model
reg = full_pipeline.fit(X_train, y_train)

In [59]:
y_train_pred = reg.predict(X_train)
#mse_train = mean_squared_error(y_train, y_train_pred)
roc_train = roc_auc_score(y_train[:, 0], y_train_pred[:, 0])
print(roc_train) #why does the ROC not match previous results that we had? Should be 0.71

0.7053562554709693


In [60]:
#data cleaning and engineering--not done
#dropping Quakers column and unnamed
#changing one of the altitude to log and droping the original
'''X_test["altitude_mean_log"] = np.log(X_test["altitude_mean_meters"])
X_test.drop(['altitude_mean_meters'], axis=1, inplace=True)
X_test.drop(['Quakers'], axis=1, inplace=True)
X_test.drop(['Unnamed: 0'], axis=1, inplace=True)
# fillna with mean.. 
X_test["altitude_low_meters"] = X_test["altitude_low_meters"].fillna(altitude_low_meters_mean)
X_test["altitude_high_meters"] = X_test["altitude_high_meters"].fillna(altitude_high_meters_mean)
X_test["altitude_mean_log"] = X_test["altitude_mean_log"].fillna(altitude_mean_log_mean)'''

'X_test["altitude_mean_log"] = np.log(X_test["altitude_mean_meters"])\nX_test.drop([\'altitude_mean_meters\'], axis=1, inplace=True)\nX_test.drop([\'Quakers\'], axis=1, inplace=True)\nX_test.drop([\'Unnamed: 0\'], axis=1, inplace=True)\n# fillna with mean.. \nX_test["altitude_low_meters"] = X_test["altitude_low_meters"].fillna(altitude_low_meters_mean)\nX_test["altitude_high_meters"] = X_test["altitude_high_meters"].fillna(altitude_high_meters_mean)\nX_test["altitude_mean_log"] = X_test["altitude_mean_log"].fillna(altitude_mean_log_mean)'

In [61]:
y_test_pred = reg.predict(X_test)
roc_test = roc_auc_score(y_test[:, 0], y_test_pred[:, 0])
print(roc_test) #looks correct

0.6908088321028654


In [62]:
#keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model)
params = {

  }

In [63]:
mlflow.log_params(params)
mlflow.set_tag("running_from_jupyter", "True") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", roc_train)
mlflow.log_metric("test -" + "ROC", roc_test)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [66]:
mlflow.get_run(run_id=run.info.run_id)

<Run: data=<RunData: metrics={'test -ROC': 0.6908088321028654, 'train -ROC': 0.7053562554709693}, params={}, tags={'mlflow.source.git.commit': '14154313dfd99736e2d1f2b02176bd92af6cd9c6',
 'mlflow.source.name': '/Users/christinarudolf/Documents/neuefische_ds/TheFluShot/.venv/lib/python3.8/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'christinarudolf',
 'running_from_jupyter': 'True'}>, info=<RunInfo: artifact_uri='s3://neuefische-mlflow/mlflow-artifacts/24/9aeea82b093c45479c462b814aa00633/artifacts', end_time=1626794684689, experiment_id='24', lifecycle_stage='active', run_id='9aeea82b093c45479c462b814aa00633', run_uuid='9aeea82b093c45479c462b814aa00633', start_time=1626794683284, status='FINISHED', user_id='christinarudolf'>>

## Checking the experiments

while the next cell is running you will not be able to run other cells in the notebook

In [65]:
#use if running locally
#!mlflow ui